In [1]:
import glob
import tqdm
import matplotlib.pyplot as plt
import numpy as np

In [101]:
filenames=glob.glob(r"C:/Users/dlara/Downloads\MODELO 3D SANTIAGO CENTRO/*/[a-f]*.obj")

In [103]:
def map_d(x, in_min, in_max, out_min, out_max):
    """Genera una interpolacion para pasar de un rango a otro"""
    return (x - in_min) * (out_max - out_min) / (in_max - in_min) + out_min

In [105]:
X=[]
Y=[]
Z=[]
res=[]
for file in tqdm.tqdm(filenames):
    with open(file,"r") as f:
        lines=f.readlines()
    for line in lines:
        if line[:2]=="v ":
            res.append(line.split()+list([file]))

100%|██████████| 15592/15592 [01:31<00:00, 169.61it/s]


In [123]:
filenames

['C:/Users/dlara/Downloads\\MODELO 3D SANTIAGO CENTRO\\c00\\a.obj',
 'C:/Users/dlara/Downloads\\MODELO 3D SANTIAGO CENTRO\\c00\\b0.obj',
 'C:/Users/dlara/Downloads\\MODELO 3D SANTIAGO CENTRO\\c00\\b1.obj',
 'C:/Users/dlara/Downloads\\MODELO 3D SANTIAGO CENTRO\\c00\\b2.obj',
 'C:/Users/dlara/Downloads\\MODELO 3D SANTIAGO CENTRO\\c00\\b3.obj',
 'C:/Users/dlara/Downloads\\MODELO 3D SANTIAGO CENTRO\\c00\\c07.obj',
 'C:/Users/dlara/Downloads\\MODELO 3D SANTIAGO CENTRO\\c00\\c16.obj',
 'C:/Users/dlara/Downloads\\MODELO 3D SANTIAGO CENTRO\\c00\\c17.obj',
 'C:/Users/dlara/Downloads\\MODELO 3D SANTIAGO CENTRO\\c00\\c25.obj',
 'C:/Users/dlara/Downloads\\MODELO 3D SANTIAGO CENTRO\\c00\\c27.obj',
 'C:/Users/dlara/Downloads\\MODELO 3D SANTIAGO CENTRO\\c00\\c34.obj',
 'C:/Users/dlara/Downloads\\MODELO 3D SANTIAGO CENTRO\\c00\\c35.obj',
 'C:/Users/dlara/Downloads\\MODELO 3D SANTIAGO CENTRO\\c00\\c36.obj',
 'C:/Users/dlara/Downloads\\MODELO 3D SANTIAGO CENTRO\\c00\\c37.obj',
 'C:/Users/dlara/Downloads

In [106]:
import pandas as pd

In [107]:
df_coords=pd.DataFrame(res)
df_coords[3]=df_coords[3].astype("float64")
df_coords[1]=df_coords[1].astype("float64")
df_coords[2]=df_coords[2].astype("float64")
df_coords=df_coords.sort_values(by=[3]).reset_index(drop=True)

In [108]:
df_coords=df_coords.drop_duplicates(subset=[1,2,3]).reset_index(drop=True)
df_coords=df_coords.astype({1:"int",2:int,3:int})
df_coords

,0,1,2,3,4
0,v,1236,1123,22,C:/Users/dlara/Downloads\MODELO 3D SANTIAGO CE...
1,v,1236,1123,22,C:/Users/dlara/Downloads\MODELO 3D SANTIAGO CE...
2,v,1236,1123,22,C:/Users/dlara/Downloads\MODELO 3D SANTIAGO CE...
3,v,1235,1123,22,C:/Users/dlara/Downloads\MODELO 3D SANTIAGO CE...
4,v,1235,1123,22,C:/Users/dlara/Downloads\MODELO 3D SANTIAGO CE...
...,...,...,...,...,...
14422199,v,849,324,241,C:/Users/dlara/Downloads\MODELO 3D SANTIAGO CE...
14422200,v,850,324,241,C:/Users/dlara/Downloads\MODELO 3D SANTIAGO CE...
14422201,v,850,323,241,C:/Users/dlara/Downloads\MODELO 3D SANTIAGO CE...
14422202,v,849,324,241,C:/Users/dlara/Downloads\MODELO 3D SANTIAGO CE...


In [111]:
max_coord=np.max(df_coords[3])
min_coord= np.min(df_coords[3])
max_coord, min_coord

(241, 22)

In [112]:
new_image_width = int(np.max(df_coords[1]))+1
new_image_height = int(np.max(df_coords[2]))+1
new_image_height,new_image_width

(1257, 1374)

In [113]:
color = (0,0,0)
result = np.full((new_image_height,new_image_width, 3), color, dtype="float32")
result=result[:,:,0]


In [114]:
import geopandas as gpd
from shapely.geometry import Point
punto_max=gpd.GeoSeries(Point([-99.772385123212771, 19.792835251567379]),crs=6364)
punto_max.crs

<Geographic 3D CRS: EPSG:6364>
Name: Mexico ITRF2008
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
- h[up]: Ellipsoidal height (metre)
Area of Use:
- name: Mexico - onshore and offshore.
- bounds: (-122.19, 12.1, -84.64, 32.72)
Datum: Mexico ITRF2008
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [13]:
print(punto_max.to_crs(3857))

0    POINT (-11106611.107 2248505.500)
dtype: geometry


In [115]:
df_coords["Z"]=df_coords[3].apply(lambda x: map_d(x,min_coord,max_coord,2508.7909335236886,2728.701065688485))
df_coords

,0,1,2,3,4,Z
0,v,1236,1123,22,C:/Users/dlara/Downloads\MODELO 3D SANTIAGO CE...,2508.790934
1,v,1236,1123,22,C:/Users/dlara/Downloads\MODELO 3D SANTIAGO CE...,2508.790934
2,v,1236,1123,22,C:/Users/dlara/Downloads\MODELO 3D SANTIAGO CE...,2508.790934
3,v,1235,1123,22,C:/Users/dlara/Downloads\MODELO 3D SANTIAGO CE...,2508.790934
4,v,1235,1123,22,C:/Users/dlara/Downloads\MODELO 3D SANTIAGO CE...,2508.790934
...,...,...,...,...,...,...
14422199,v,849,324,241,C:/Users/dlara/Downloads\MODELO 3D SANTIAGO CE...,2728.701066
14422200,v,850,324,241,C:/Users/dlara/Downloads\MODELO 3D SANTIAGO CE...,2728.701066
14422201,v,850,323,241,C:/Users/dlara/Downloads\MODELO 3D SANTIAGO CE...,2728.701066
14422202,v,849,324,241,C:/Users/dlara/Downloads\MODELO 3D SANTIAGO CE...,2728.701066


In [116]:
import tqdm
for i in tqdm.tqdm(range(len(df_coords))):
    result[df_coords.loc[i,2],df_coords.loc[i,1]]=df_coords.loc[i,3]
    

100%|██████████| 14422204/14422204 [06:25<00:00, 37368.89it/s]


In [117]:
result.max()
# result.min()

241.0

In [6]:
with open("E:/aeromodelismo/MODELO_3D_SANTIAGO_NOROESTE/doc.kml","r") as kml:
    li=kml.readlines()
lineas_utiles=[]
for l in li:
    l=l.replace("\n","")
    lineas_utiles.append(l)
    if l.find("href")>=0:
        break
lineas_utiles

['<?xml version="1.0" encoding="UTF-8"?>',
 '<kml xmlns="http://www.opengis.net/kml/2.2" hint="quadtree">',
 '  <Folder>',
 '    <Region>',
 '      <LatLonAltBox>',
 '        <west>-99.782989391660777</west>',
 '        <east>-99.767642320395964</east>',
 '        <south>19.788728569098481</south>',
 '        <north>19.796121214723037</north>',
 '        <minAltitude>2540.1094647646382</minAltitude>',
 '        <maxAltitude>2593.3428956753705</maxAltitude>',
 '        <altitudeMode>absolute</altitudeMode>',
 '      </LatLonAltBox>',
 '      <Lod>',
 '        <minLodPixels>200</minLodPixels>',
 '        <maxLodPixels>512</maxLodPixels>',
 '      </Lod>',
 '    </Region>',
 '    <Placemark>',
 '      <Model>',
 '        <altitudeMode>absolute</altitudeMode>',
 '        <Location>',
 '          <longitude>-99.784036190795319</longitude>',
 '          <latitude>19.788399159150078</latitude>',
 '          <altitude>2531.0369093063096</altitude>',
 '        </Location>',
 '        <Link>',
 

In [118]:
def map_d(x, in_min, in_max, out_min, out_max):
    """Genera una interpolacion para pasar de un rango a otro"""
    return (x - in_min) * (out_max - out_min) / (in_max - in_min) + out_min


In [ ]:
metrosxpixel longitud de array/ diferencia entre west y east en crs:3857

In [121]:
# Cambié valores latlon , basandome en kml, al crs 3857 para sacar la diferencia entre west y east
# Para la longitud es max lenght
(1257)/(-11105244.964 - (-11106611.107))
# (1257)/(-99.76011285106506 - (-99.772385123212771))


0.9201086562673154

In [173]:

#  (-11106611.107 2248505.500)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [122]:
import rasterio
from affine import Affine
##extremo superior izquierdo

longitud=-11106611.107
latitud=2248505.500
metrosxpixel=0.9201086562673154
crs=3857
matriz=np.flip(result, 0)


transform=Affine(metrosxpixel,0,longitud,0,-metrosxpixel,latitud) ## 0 representa las componetes de direccion de la coordenada
out_meta=({"driver": "GTiff",
            "height": matriz.shape[0],
            "width": matriz.shape[1],
            "count":1,
            "dtype":np.float32,
            "crs":crs,
          'transform':transform})
with rasterio.open(r"D:\blac1k2.tif",'w', **out_meta) as dest_file:
    dest_file.write(matriz, 1)